In [76]:
import sys
sys.path.append("../") # go to parent dir

from pprint import pprint
from pathlib import Path
import pyjson5

from dcf.dls import make_particle_sizer
from dcf.utils import relative_error, average_dict_val

project_path = Path("..").resolve()

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
dls_data_path = project_path / "data" / "DLS" / "dls_results.json5"
particle_data_path = project_path / "data" / "particle_data.json5"

with open(dls_data_path) as f:
    dls_data = pyjson5.load(f)

with open(particle_data_path) as f:
    particle_data = pyjson5.load(f)


In [78]:
# Normalise data
for exp_name, exp in dls_data.items():
    exp |= particle_data.get(exp["particle"], {})


In [79]:
# Calculate diameter
viscosity = 0.9775e-3
temperature = 273 + 21
wavelength = 635.0e-9
refractive_index = 1.333

particle_sizer = make_particle_sizer(viscosity=viscosity, temperature=temperature, wavelength=wavelength, refractive_index=refractive_index)

for exp_name, exp in dls_data.items():
    exp["experimental_diameter"] = particle_sizer(exp["tau"], exp["theta"])

    if "known_diameter" in exp:
        exp["error"] = relative_error(exp["known_diameter"], exp["experimental_diameter"])

pprint(dls_data)

{'2A25_01': {'concentration': 0.1,
             'error': -0.5945715887619231,
             'experimental_diameter': 8.554539477123423e-08,
             'known_diameter': 2.11e-07,
             'material_type': 'SiO2',
             'particle': 'A',
             'std_dev_diameter': 8e-09,
             'tau': 0.0059559,
             'theta': 25},
 '2A30_0001': {'concentration': 0.001,
               'error': -0.014873498292861009,
               'experimental_diameter': 2.0786169186020632e-07,
               'known_diameter': 2.11e-07,
               'material_type': 'SiO2',
               'particle': 'A',
               'std_dev_diameter': 8e-09,
               'tau': 0.0101206,
               'theta': 30},
 '2A30_001': {'concentration': 0.01,
              'error': -0.17966799342063844,
              'experimental_diameter': 1.7309005338824528e-07,
              'known_diameter': 2.11e-07,
              'material_type': 'SiO2',
              'particle': 'A',
              'std_dev_diame

In [80]:
# Mystery particle
mystery_particles = {k: v for k, v in dls_data.items() if k.startswith("2M")}
mystery_average = average_dict_val(mystery_particles, "experimental_diameter")

print(f"Mystery average: {mystery_average}")
print(f"Mystery knownish: {particle_data['C']['known_diameter']}")
print(f"Mystery error: {relative_error(particle_data['C']['known_diameter'], mystery_average)}")



Mystery average: 5.269795730550598e-07
Mystery knownish: 5.18e-07
Mystery error: 0.01733508311787615


In [81]:
# A with varying angles
a_particles = {k: v for k, v in dls_data.items() if k.startswith("2A") and k.endswith("_01")}
a_average = average_dict_val(a_particles, "experimental_diameter")

a_error = relative_error(particle_data["A"]["known_diameter"], a_average)

print(f"A average: {a_average}")
print(f"A known: {particle_data['A']['known_diameter']}")
print(f"A error: {relative_error(particle_data['A']['known_diameter'], a_average)}")

A average: 1.7881184013162757e-07
A known: 2.11e-07
A error: -0.15255052070318684
